In [1]:
import requests
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import csv
import time
import json
from pandas import json_normalize

In [2]:
'''
1. LH 단지정보
지역본부명, 세대수  -> '대구경북지역본부' 지역 쪼개기
지역본부명-> 지역으로 통일
공급유형 -> '국민임대', '공공임대(50년)', '영구임대', '임대상가', '공공임대(10년)', '공공임대(분납)',
       '장기전세', '공공분양', '행복주택', '공공임대(5년)'으로 분류

-새로운 데이터셋1 -
컬럼 
지역, 공급유형, 총세대수, 단지명, 주소 
# 인천이 어디에 속하는지 알 필요가 있음


2. train 
새로운 데이터셋1과 지역, 총세대수, 공급유형  공통값을 필터링

-새로운 데이터셋2 -
컬럼
단지코드, 지역, 공급유형, 총세대수, 단지명, 주소


3. 단지별 연령대별 성별정보
주택유형 필터['아파트']-> 데이터프레임1 생성
데이터프레임1에 도로명주소 str 필터 -> 지역 컬럼구분
지역, 세대수를 공통필터 -> 세대수구성 정보 획득


-새로운 데이터셋3 -
컬럼
단지코드, 지역, 공급유형, 총세대수, 단지명, 주소, 세대수구성원


4. 다시 트레인에 병합
단지코드를 기준으로 세대수구성원 병합
'''

"\n1. LH 단지정보\n지역본부명, 세대수  -> '대구경북지역본부' 지역 쪼개기\n지역본부명-> 지역으로 통일\n\n-새로운 데이터셋1 -\n컬럼 \n지역, 총세대수, 단지명, 주소 \n# 인천이 어디에 속하는지 알 필요가 있음\n\n\n2. train \n새로운 데이터셋1과 지역, 총세대수 공통값을 필터링\n\n-새로운 데이터셋2 -\n컬럼\n단지코드, 지역, 총세대수, 단지명, 주소\n\n\n3. 단지별 연령대별 성별정보\n주택유형 필터['아파트']-> 데이터프레임1 생성\n데이터프레임1에 도로명주소 str 필터 -> 지역 컬럼구분\n지역, 세대수를 공통필터 -> 세대수구성 정보 획득\n\n\n-새로운 데이터셋3 -\n컬럼\n단지코드, 지역, 총세대수, 단지명, 주소, 세대수구성원\n\n\n4. 다시 트레인에 병합\n단지코드를 기준으로 세대수구성원 병합\n\n-최종 데이터프레임-\n"

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
danji = pd.read_csv('공공데이터/한국토지주택공사_전국 LH아파트 단지정보_20210430.csv', encoding='cp949')
gender = pd.read_csv('공공데이터/한국토지주택공사_임대주택 단지별 연령대별 성별정보_20210511.csv', encoding='cp949')
display(train)
display(test)
display(danji)
display(gender)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,아파트,강원도,국민임대,49.20,19,7.0,A,11346000,116090,0.0,1.0,166.0,146.0
2948,C2532,239,아파트,강원도,국민임대,51.08,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2949,C2532,239,아파트,강원도,국민임대,51.73,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2950,C2532,239,아파트,강원도,국민임대,51.96,114,7.0,A,14005000,142310,0.0,1.0,166.0,146.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,L,-,-,0.0,1.0,467.0
1018,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,H,6872000,106400,0.0,2.0,300.0
1019,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,H,6872000,106400,0.0,2.0,300.0
1020,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,H,13410000,144600,0.0,2.0,300.0


,지역본부명,단지코드,단지명,공급유형,주소,세대수,동수
0,서울지역본부,C02012,(NHF제4호)하남미사A4,공공임대(10년),"경기도 하남시 미사강변북로 25(선동) 경기 하남시 망월동, 풍산동, 선동, 덕풍동...",996,10
1,서울지역본부,C00458,가평읍내,공공임대(5년),NaN,355,5
2,서울지역본부,C00037,가평읍내,공공임대(50년),경기도 가평군 가평읍 가화로 142-26(가평읍내주공아파트),355,5
3,서울지역본부,C01592,가평읍내2단지,국민임대,경기도 가평군 가평읍 가화로 164 (휴먼시아아파트),335,6
4,서울지역본부,889,강남4BL,공공분양+토지임대부분양,서울특별시 강남구 자곡로3길 45(자곡동618),402,0
...,...,...,...,...,...,...,...
3658,세종특별본부,C02151,행정중심복합도시 3-1생활권 M5블록,공공임대(10년),세종특별자치시 대평로 56(대평동),1438,18
3659,세종특별본부,C02218,행정중심복합도시 3-2생활권 M2블록,국민임대,"세종특별자치시 보람1길 12(보람동,호려울마을2단지)",674,5
3660,세종특별본부,C02217,행정중심복합도시 3-3(M6)BL,공공분양,"세종특별자치시 남세종로 302(소담동,새샘마을6단지)",0,19
3661,세종특별본부,C02550,행정중심복합도시 3-3생활권 M2블록,국민임대,NaN,1100,4


,단지_일련번호,임대주택유형,공급기관명,주택명,주택유형,도로명주소,총세대수,10대미만(여자),10대미만(남자),10대(여자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,30485277,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로102번길 19,9.0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1,30485278,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로102번길 7-2,8.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,30485279,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로102번길 7-4,7.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,30485280,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로52번길 9-11,10.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,30485281,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로74번길 19,8.0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19450,30855821,매입임대,NaN,매입다가구(전북김제시),다가구주택,NaN,0.0,1,4,2,...,0,0,1,0,0,0,0,0,0,0
19451,30855838,매입임대,NaN,매입다가구(전북남원시),다가구주택,NaN,0.0,0,1,2,...,0,0,0,1,0,1,1,0,0,0
19452,30855839,매입임대,NaN,매입다가구(전북남원시),다가구주택,NaN,0.0,1,3,2,...,2,2,0,0,1,1,0,0,0,0
19453,31284963,건설임대,NaN,양주옥정 A-20(2)(공임리츠),아파트,NaN,619.0,99,114,134,...,30,36,9,9,12,0,0,0,0,0


In [4]:
danji['지역본부명'].unique()

array(['서울지역본부', '강원지역본부', '경기지역본부', '인천지역본부', '충북지역본부', '대전충남지역본부',
       '전북지역본부', '광주전남지역본부', '대구경북지역본부', '경남지역본부', '부산울산지역본부', '제주지역본부',
       '세종특별본부'], dtype=object)

In [5]:
train['지역'].unique()

array(['경상북도', '경상남도', '대전광역시', '경기도', '전라북도', '강원도', '광주광역시', '충청남도',
       '부산광역시', '제주특별자치도', '울산광역시', '충청북도', '전라남도', '대구광역시', '서울특별시',
       '세종특별자치시'], dtype=object)

In [6]:
a = test['지역'].unique()
display(a)

array(['경기도', '부산광역시', '전라북도', '경상남도', '충청남도', '대전광역시', '제주특별자치도', '강원도',
       '울산광역시', '경상북도', '충청북도', '광주광역시', '전라남도', '대구광역시', '세종특별자치시'],
      dtype=object)

In [7]:
train['공급유형'].unique()

array(['국민임대', '공공임대(50년)', '영구임대', '임대상가', '공공임대(10년)', '공공임대(분납)',
       '장기전세', '공공분양', '행복주택', '공공임대(5년)'], dtype=object)

In [8]:
test['공급유형'].unique()

array(['국민임대', '영구임대', '임대상가', '공공임대(50년)', '공공임대(10년)', '공공임대(분납)',
       '행복주택'], dtype=object)

In [9]:
danji['공급유형'].unique()

array(['공공임대(10년)', '공공임대(5년)', '공공임대(50년)', '국민임대', '공공분양+토지임대부분양',
       '공공분양', '분양주택', '국민임대+영구임대', '행복주택', '공공분양+공공임대(5년)', '행복주택+임대상가',
       nan, '공공임대(분납)', '영구임대+임대상가', '공공임대(10년)+공공임대(분납)',
       '장기전세+국민임대+영구임대', '공공분양+공공임대(50년)', '영구임대', '외인임대',
       '공공분양+공공임대(10년)', '후분양', '토지임대부분양', '임대상가', '공공임대(10년)+임대상가',
       '공공임대(50년)+영구임대', '국민임대+행복주택'], dtype=object)

In [10]:
# 트레인과 테스트에 공급유형 결측이 없으므로, 결측이있는 단지데이터는 쓰지 않았다고 가정한다.
danji.isna().sum()

지역본부명      0
단지코드       0
단지명        1
공급유형     142
주소       773
세대수        0
동수         0
dtype: int64

In [11]:
# 결측치 대부분 입주자 모집중인 아파트
danji.isna().sum()
danji1 = danji.dropna(subset=['주소'])

In [12]:
경기 = danji1[danji1['주소'].str.contains('경기')]
경기['지역본부명'] = '경기도'

서울 = danji1[danji1['주소'].str.contains('서울')]
서울['지역본부명'] = '서울특별시'

충남 = danji1[danji1['주소'].str.contains('충청남도|충남')]
충남['지역본부명'] = '충청남도'

충북 = danji1[danji1['주소'].str.contains('충청북도|충북')]
충북['지역본부명'] = '충청북도'

대전 = danji1[danji1['주소'].str.contains('대전')]
대전['지역본부명'] = '대전광역시'

광주 = danji1[danji1['주소'].str.contains('광주')]
광주['지역본부명'] = '광주광역시'

전남 = danji1[danji1['주소'].str.contains('전라남도|전남')]
전남['지역본부명'] = '전라남도'

전북 = danji1[danji1['주소'].str.contains('전라북도|전북')]
전북['지역본부명'] = '전라북도'

부산 = danji1[danji1['주소'].str.contains('부산')]
부산['지역본부명'] = '부산광역시'

울산 = danji1[danji1['주소'].str.contains('울산')]
울산['지역본부명'] = '울산광역시'

대구 = danji1[danji1['주소'].str.contains('대구')]
대구['지역본부명'] = '대구광역시'

경북 = danji1[danji1['주소'].str.contains('경상북도|경북')]
경북['지역본부명'] = '경상북도'

경남 = danji1[danji1['주소'].str.contains('경상남도|경남')]
경남['지역본부명'] = '경상남도'

강원 = danji1[danji1['주소'].str.contains('강원도|강원')]
강원['지역본부명'] = '강원도'

대구 = danji1[danji1['주소'].str.contains('대구')]
대구['지역본부명'] = '대구광역시'

제주 = danji1[danji1['주소'].str.contains('제주')]
제주['지역본부명'] = '제주특별자치도'

세종 = danji1[danji1['주소'].str.contains('세종')]
세종['지역본부명'] = '세종특별자치시'

temp1 = pd.concat([경기, 서울, 충남, 충북, 대전, 광주, 전남, 전북, 부산, 울산, 대구, 경북, 경남, 강원, 대구, 제주, 세종])
temp1.drop(['단지코드'], axis=1, inplace=True)
temp1.rename(columns={'지역본부명':'지역', '세대수':'총세대수'}, inplace=True)
temp1.reset_index(drop = True, inplace=True)
temp1

<ipython-input-12-374a0e03cdbd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  경기['지역본부명'] = '경기도'
<ipython-input-12-374a0e03cdbd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  서울['지역본부명'] = '서울특별시'
<ipython-input-12-374a0e03cdbd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

,지역,단지명,공급유형,주소,총세대수,동수
0,경기도,(NHF제4호)하남미사A4,공공임대(10년),"경기도 하남시 미사강변북로 25(선동) 경기 하남시 망월동, 풍산동, 선동, 덕풍동...",996,10
1,경기도,가평읍내,공공임대(50년),경기도 가평군 가평읍 가화로 142-26(가평읍내주공아파트),355,5
2,경기도,가평읍내2단지,국민임대,경기도 가평군 가평읍 가화로 164 (휴먼시아아파트),335,6
3,경기도,고양 햇빛마을(중형임대),공공임대(5년),경기 고양시 덕양구 행신동 958(21/12) 햇빛마을,1,1
4,경기도,고양관산1,공공분양,경기 고양시 덕양구 관산동 996번지 주공그린빌,0,16
...,...,...,...,...,...,...
2843,세종특별자치시,행정중심복합도시 2-2(M8)BL,국민임대,"세종특별자치시 새롬중앙로 20(새롬동,새뜸마을8단지)",906,5
2844,세종특별자치시,행정중심복합도시 3-1생활권 M5블록,공공임대(10년),세종특별자치시 대평로 56(대평동),1438,18
2845,세종특별자치시,행정중심복합도시 3-2생활권 M2블록,국민임대,"세종특별자치시 보람1길 12(보람동,호려울마을2단지)",674,5
2846,세종특별자치시,행정중심복합도시 3-3(M6)BL,공공분양,"세종특별자치시 남세종로 302(소담동,새샘마을6단지)",0,19


In [13]:
국민 = temp1[temp1['공급유형'].str.contains('국민임대', na=False)]
국민['공급유형'] = '국민임대'

상가 = temp1[temp1['공급유형'].str.contains('임대상가', na=False)]
상가['공급유형'] = '임대상가'

영구 = temp1[temp1['공급유형'].str.contains('영구임대', na=False)]
영구['공급유형'] = '영구임대'

분납 = temp1[temp1['공급유형'].str.contains('공공임대(분납)', na=False)]
분납['공급유형'] = '공공임대(분납)'

오 = temp1[temp1['공급유형'].str.contains('공공임대(5년)', na=False)]
오['공급유형'] = '공공임대(5년)'

십 = temp1[temp1['공급유형'].str.contains('공공임대(10년)', na=False)]
십['공급유형'] = '공공임대(10년)'

오십 = temp1[temp1['공급유형'].str.contains('공공임대(50년)', na=False)]
오십['공급유형'] = '공공임대(50년)'

분양 = temp1[temp1['공급유형'].str.contains('공공분양', na=False)]
분양['공급유형'] = '공공분양'

전세 = temp1[temp1['공급유형'].str.contains('장기전세', na=False)]
전세['공급유형'] = '장기전세'

행복 = temp1[temp1['공급유형'].str.contains('행복주택', na=False)]
행복['공급유형'] = '행복주택'



temp2 = pd.concat([국민, 상가, 영구, 분납, 오, 십, 오십, 분양, 전세, 행복])
temp2.reset_index(drop = True, inplace=True)
temp2

<ipython-input-13-005d321b9b10>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  국민['공급유형'] = '국민임대'
<ipython-input-13-005d321b9b10>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  상가['공급유형'] = '임대상가'
<ipython-input-13-005d321b9b10>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

,지역,단지명,공급유형,주소,총세대수,동수
0,경기도,가평읍내2단지,국민임대,경기도 가평군 가평읍 가화로 164 (휴먼시아아파트),335,6
1,경기도,구리갈매1단지,국민임대,경기도 구리시 갈매순환로 143(갈매동) 구리갈매1단지,1546,8
2,경기도,남양주가운1,국민임대,"경기도 남양주시 가운로2길 28(다산동,남양주가운마을휴먼시아1단지)",1417,16
3,경기도,남양주가운2,국민임대,"경기도 남양주시 가운로2길 8(다산동,남양주가운마을휴먼시아2단지)",388,4
4,경기도,남양주마석,국민임대,경기도 남양주시 화도읍 맷돌로91번길 7,488,5
...,...,...,...,...,...,...
2164,제주특별자치도,서귀포서홍,행복주택,제주특별자치도 서귀포시 장수로 115(서홍동) 서귀포서홍행복주택,90,1
2165,제주특별자치도,서귀포혁신도시 LH3단지,행복주택,제주특별자치도 서귀포시 서호중앙로 56-13(서호동) 서귀포혁신도시 LH3단지,202,2
2166,제주특별자치도,제주봉개,행복주택,제주특별자치도 제주시 번영로 528(봉개동),540,7
2167,세종특별자치시,세종서창 행복주택 A-1,행복주택,세종특별자치시 조치원읍 조치원중고길 7 세종서창 행복주택,453,4


In [14]:
merge_df1 = pd.merge(train, temp2, on=['지역', '공급유형', '총세대수'])
merge_df1 = merge_df1[['단지코드', '지역', '공급유형', '단지명', '총세대수', '주소']]
display(merge_df1)

merge_df2 = pd.merge(test, temp2, on=['지역', '공급유형', '총세대수'])
merge_df2 = merge_df2[['단지코드', '지역', '공급유형', '단지명', '총세대수', '주소']]
display(merge_df2)
# merge_df1.to_csv('LH 단지정보+트레인.csv', encoding='cp949')

,단지코드,지역,공급유형,단지명,총세대수,주소
0,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)"
1,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)"
2,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)"
3,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)"
4,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)"
...,...,...,...,...,...,...
2863,C2532,강원도,국민임대,"평창하진부(07,매입) 1단지",239,강원도 평창군 진부면 경강로 3587(평창하진부주공아파트)
2864,C2532,강원도,국민임대,평창하진부,239,강원도 평창군 진부면 경강로 3587 (평창하진부주공아파트)
2865,C2532,강원도,국민임대,"평창하진부(07,매입) 1단지",239,강원도 평창군 진부면 경강로 3587(평창하진부주공아파트)
2866,C2532,강원도,국민임대,평창하진부,239,강원도 평창군 진부면 경강로 3587 (평창하진부주공아파트)


,단지코드,지역,공급유형,단지명,총세대수,주소
0,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)"
1,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)"
2,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)"
3,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)"
4,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)"
...,...,...,...,...,...,...
980,C1683,경기도,행복주택,평택청북 B12BL 행복주택,328,경기도 평택시 청북읍 안청로1길 21(평택청북 LH B12단지)
981,C2152,강원도,영구임대,정선신동 국민임대,120,강원도 정선군 신동읍 새골길 110 정신신동
982,C2152,강원도,영구임대,화천신읍(공공실버) 영구임대,120,강원도 화천군 화천읍 산천어길 196 화천신읍 공공실버주택
983,C2152,강원도,영구임대,정선신동 국민임대,120,강원도 정선군 신동읍 새골길 110 정신신동


In [15]:
# merge_df1.to_csv('LH 단지정보+트레인.csv', encoding='cp949')
# merge_df2.to_csv('LH 단지정보+테스트.csv', encoding='cp949')

In [16]:
gender_apt = gender[gender['주택유형']=='아파트']
gender_apt.insert(2, '지역', '-1')

경기 = gender_apt[gender_apt['도로명주소'].str.contains('경기', na=False)]
경기['지역'] = '경기도'

서울 = gender_apt[gender_apt['도로명주소'].str.contains('서울', na=False)]
서울['지역'] = '서울특별시'

충남 = gender_apt[gender_apt['도로명주소'].str.contains('충청남도|충남', na=False)]
충남['지역'] = '충청남도'

충북 = gender_apt[gender_apt['도로명주소'].str.contains('충청북도|충북', na=False)]
충북['지역'] = '충청북도'

대전 = gender_apt[gender_apt['도로명주소'].str.contains('대전', na=False)]
대전['지역'] = '대전광역시'

광주 = gender_apt[gender_apt['도로명주소'].str.contains('광주', na=False)]
광주['지역'] = '광주광역시'

전남 = gender_apt[gender_apt['도로명주소'].str.contains('전라남도|전남', na=False)]
전남['지역'] = '전라남도'

전북 = gender_apt[gender_apt['도로명주소'].str.contains('전라북도|전북', na=False)]
전북['지역'] = '전라북도'

부산 = gender_apt[gender_apt['도로명주소'].str.contains('부산', na=False)]
부산['지역'] = '부산광역시'

울산 = gender_apt[gender_apt['도로명주소'].str.contains('울산', na=False)]
울산['지역'] = '울산광역시'

대구 = gender_apt[gender_apt['도로명주소'].str.contains('대구', na=False)]
대구['지역'] = '대구광역시'

경북 = gender_apt[gender_apt['도로명주소'].str.contains('경상북도|경북', na=False)]
경북['지역'] = '경상북도'

경남 = gender_apt[gender_apt['도로명주소'].str.contains('경상남도|경남', na=False)]
경남['지역'] = '경상남도'

강원 = gender_apt[gender_apt['도로명주소'].str.contains('강원도|강원', na=False)]
강원['지역'] = '강원도'

대구 = gender_apt[gender_apt['도로명주소'].str.contains('대구', na=False)]
대구['지역'] = '대구광역시'

제주 = gender_apt[gender_apt['도로명주소'].str.contains('제주', na=False)]
제주['지역'] = '제주특별자치도'

세종 = gender_apt[gender_apt['도로명주소'].str.contains('세종', na=False)]
세종['지역'] = '세종특별자치시'

temp3 = pd.concat([경기, 서울, 충남, 충북, 대전, 광주, 전남, 전북, 부산, 울산, 대구, 경북, 경남, 강원, 대구, 제주, 세종])
temp3

<ipython-input-16-93b65d611efc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  경기['지역'] = '경기도'
<ipython-input-16-93b65d611efc>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  서울['지역'] = '서울특별시'
<ipython-input-16-93b65d611efc>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

,단지_일련번호,임대주택유형,지역,공급기관명,주택명,주택유형,도로명주소,총세대수,10대미만(여자),10대미만(남자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
21,4181,건설임대,경기도,성남시,선경단대영구임대아파트,아파트,경기도 성남시 수정구 산성대로383번길 28,60.0,0,0,...,17,8,6,5,4,6,3,0,0,0
188,30582333,건설임대,경기도,SH공사,수락리버시티1단지,아파트,경기도 의정부시 누원로 51,680.0,21,11,...,109,90,65,46,19,15,14,5,0,0
189,30582334,건설임대,경기도,SH공사,수락리버시티2단지,아파트,경기도 의정부시 누원로 52,473.0,13,9,...,27,36,16,15,12,5,7,2,1,1
2795,88058,매입임대,경기도,경기도시공사,NaN,아파트,경기도 광명시 목감로 58,112.0,9,10,...,21,10,2,7,3,0,1,0,0,0
2800,30819467,건설임대,경기도,경기도시공사,호수마을자연앤이편한세상,아파트,경기도 김포시 김포한강4로420번길 164,559.0,53,60,...,65,54,29,27,8,4,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19351,30701530,매입임대(단지유형),세종특별자치시,LH세종,세종영조정보마을,아파트,세종특별자치시 장군면 대학길 118,5.0,0,0,...,0,0,0,0,0,0,0,0,0,0
19352,30706817,매입임대(단지유형),세종특별자치시,LH세종,조치원 신동아파밀리에 3차아파트,아파트,세종특별자치시 조치원읍 도장말길 30,37.0,0,2,...,1,1,1,0,0,0,0,0,0,0
19353,30706818,매입임대(단지유형),세종특별자치시,LH세종,조치원 신동아파밀리에 3차아파트,아파트,세종특별자치시 조치원읍 도장말길 30,37.0,2,0,...,2,4,1,0,0,0,0,0,0,0
19354,30706820,매입임대(단지유형),세종특별자치시,LH세종,조치원 신동아파밀리에 3차아파트,아파트,세종특별자치시 조치원읍 도장말길 30,37.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
temp3.columns

Index(['단지_일련번호', '임대주택유형', '지역', '공급기관명', '주택명', '주택유형', '도로명주소', '총세대수',
       '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)'],
      dtype='object')

In [18]:
merge_df_res1 = pd.merge(merge_df1, temp3, on=['지역', '총세대수'])
merge_df_res1 = merge_df_res1[['단지코드', '지역', '공급유형', '단지명', '총세대수', '주소',  '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)']]
display(merge_df_res1)

merge_df_res2 = pd.merge(merge_df2, temp3, on=['지역', '총세대수'])
merge_df_res2 = merge_df_res2[['단지코드', '지역', '공급유형', '단지명', '총세대수', '주소',  '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)']]
display(merge_df_res2)

,단지코드,지역,공급유형,단지명,총세대수,주소,10대미만(여자),10대미만(남자),10대(여자),10대(남자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)",83,111,129,134,...,223,127,77,70,40,20,2,3,0,0
1,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)",83,111,129,134,...,223,127,77,70,40,20,2,3,0,0
2,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)",83,111,129,134,...,223,127,77,70,40,20,2,3,0,0
3,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)",83,111,129,134,...,223,127,77,70,40,20,2,3,0,0
4,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)",83,111,129,134,...,223,127,77,70,40,20,2,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2618,C2532,강원도,국민임대,"평창하진부(07,매입) 1단지",239,강원도 평창군 진부면 경강로 3587(평창하진부주공아파트),18,19,40,52,...,52,39,39,20,21,7,6,4,0,0
2619,C2532,강원도,국민임대,평창하진부,239,강원도 평창군 진부면 경강로 3587 (평창하진부주공아파트),18,19,40,52,...,52,39,39,20,21,7,6,4,0,0
2620,C2532,강원도,국민임대,"평창하진부(07,매입) 1단지",239,강원도 평창군 진부면 경강로 3587(평창하진부주공아파트),18,19,40,52,...,52,39,39,20,21,7,6,4,0,0
2621,C2532,강원도,국민임대,평창하진부,239,강원도 평창군 진부면 경강로 3587 (평창하진부주공아파트),18,19,40,52,...,52,39,39,20,21,7,6,4,0,0


,단지코드,지역,공급유형,단지명,총세대수,주소,10대미만(여자),10대미만(남자),10대(여자),10대(남자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)",50,51,112,122,...,214,171,87,77,40,26,5,5,1,0
1,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)",50,51,112,122,...,214,171,87,77,40,26,5,5,1,0
2,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)",50,51,112,122,...,214,171,87,77,40,26,5,5,1,0
3,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)",50,51,112,122,...,214,171,87,77,40,26,5,5,1,0
4,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)",50,51,112,122,...,214,171,87,77,40,26,5,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,C1841,광주광역시,행복주택,광주첨단H-1(창업지원),100,광주광역시 광산구 첨단중앙로152번길 6-8(쌍암동),0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
980,C1841,광주광역시,행복주택,광주첨단H-1(창업지원),100,광주광역시 광산구 첨단중앙로152번길 6-8(쌍암동),0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
981,C1841,광주광역시,행복주택,광주첨단H-2,100,광주광역시 광산구 임방울대로825번길 9(쌍암동),1,1,0,0,...,1,1,3,0,0,0,0,0,0,0
982,C1841,광주광역시,행복주택,광주첨단H-2,100,광주광역시 광산구 임방울대로825번길 9(쌍암동),0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# merge_df_res1 = merge_df_res1[['단지코드', '단지명', '주소', '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
#        '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
#        '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
#        '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)']]

# merge_df_res2 = merge_df_res2[['단지코드', '단지명', '주소', '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
#        '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
#        '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
#        '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)']]

# display(merge_df_res1)
# display(merge_df_res2)

In [20]:
merge_df_res1 = merge_df_res1[['단지코드', '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)']]

merge_df_res2 = merge_df_res2[['단지코드', '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)']]

display(merge_df_res1)
display(merge_df_res2)

,단지코드,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C2483,83,111,129,134,112,116,124,104,150,...,223,127,77,70,40,20,2,3,0,0
1,C2483,83,111,129,134,112,116,124,104,150,...,223,127,77,70,40,20,2,3,0,0
2,C2483,83,111,129,134,112,116,124,104,150,...,223,127,77,70,40,20,2,3,0,0
3,C2483,83,111,129,134,112,116,124,104,150,...,223,127,77,70,40,20,2,3,0,0
4,C2483,83,111,129,134,112,116,124,104,150,...,223,127,77,70,40,20,2,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2618,C2532,18,19,40,52,51,64,27,27,62,...,52,39,39,20,21,7,6,4,0,0
2619,C2532,18,19,40,52,51,64,27,27,62,...,52,39,39,20,21,7,6,4,0,0
2620,C2532,18,19,40,52,51,64,27,27,62,...,52,39,39,20,21,7,6,4,0,0
2621,C2532,18,19,40,52,51,64,27,27,62,...,52,39,39,20,21,7,6,4,0,0


,단지코드,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C1072,50,51,112,122,125,144,96,119,172,...,214,171,87,77,40,26,5,5,1,0
1,C1072,50,51,112,122,125,144,96,119,172,...,214,171,87,77,40,26,5,5,1,0
2,C1072,50,51,112,122,125,144,96,119,172,...,214,171,87,77,40,26,5,5,1,0
3,C1072,50,51,112,122,125,144,96,119,172,...,214,171,87,77,40,26,5,5,1,0
4,C1072,50,51,112,122,125,144,96,119,172,...,214,171,87,77,40,26,5,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,C1841,0,0,0,0,23,12,10,18,4,...,0,0,0,0,0,0,0,0,0,0
980,C1841,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
981,C1841,1,1,0,0,17,15,10,14,1,...,1,1,3,0,0,0,0,0,0,0
982,C1841,0,0,0,0,23,12,10,18,4,...,0,0,0,0,0,0,0,0,0,0


In [21]:
merge_df_res1.to_csv('트레인_단지_세대원_재병합이전.csv', encoding='cp949')
merge_df_res2.to_csv('테스트_단지_세대원_재병합이전.csv', encoding='cp949')

In [ ]:
# result_train = pd.merge(train, merge_df_res1, on=['단지코드'])
# result_test = pd.merge(test, merge_df_res2, on=['단지코드'])
# display(result_train)
# display(result_test)

In [ ]:
# result_train.to_csv('data/합친데이터_트레인_최종20210625.csv')
# result_test.to_csv('data/합친데이터_테스트_최종20210625.csv')